https://api.blockchain.info/charts/market-price?format=csv

 retrieve the dates and values for Bitcoin over the most
recent year as of when you read this. (For that reason, your results will look different
from mine, even if you use the same code.) Once you have retrieved this data, I want
you to produce a report showing

 The closing price for the most recent trading day

 The lowest historical price and the date of that price

 The highest historical price and the date of that price


In [13]:
import pandas as pd
from pandas import DataFrame, Series

In [22]:
df = pd.read_csv('https://api.blockchain.info/charts/market-price?format=csv', header=None, 
                 names=['date', 'value'])

In [23]:
df.tail(1)['value']

365    117128.97
Name: value, dtype: float64

In [24]:
df.tail(1)[['value']]

,value
365,117128.97


In [25]:
df.loc[df['value'] == df['value'].min()]

,date,value
22,2024-10-11 00:00:00,60269.24


In [26]:
df.loc[df['value'] == df['value'].max()]

,date,value
329,2025-08-14 00:00:00,123359.45


In [27]:
df

,date,value
0,2024-09-19 00:00:00,61683.91
1,2024-09-20 00:00:00,62938.20
2,2024-09-21 00:00:00,63213.19
3,2024-09-22 00:00:00,63377.01
4,2024-09-23 00:00:00,63626.42
...,...,...
361,2025-09-15 00:00:00,115384.69
362,2025-09-16 00:00:00,115402.40
363,2025-09-17 00:00:00,116797.50
364,2025-09-18 00:00:00,116453.08


In [29]:
df.set_index('date').idxmax()


value    2025-08-14 00:00:00
dtype: object

In [31]:
df.set_index('date').idxmin()
df.set_index('date').idxmax()

value    2025-08-14 00:00:00
dtype: object

In [32]:
df.set_index('date').agg(['idxmin', 'idxmax'])

,value
idxmin,2024-10-11 00:00:00
idxmax,2025-08-14 00:00:00


In [38]:
df.loc[df['date'] == df.set_index('date').idxmax()['value']]

,date,value
329,2025-08-14 00:00:00,123359.45


In [39]:
df.loc[df['date'] == df.set_index('date').idxmin()['value']]

,date,value
22,2024-10-11 00:00:00,60269.24


### Without assigning the downloaded data to an interim variable, can you return the current value? Your solution should consist of a single line of code that includes the download, selection, and calculation.

In [40]:
pd.read_csv('https://api.blockchain.info/charts/market-price?format=csv', header=None, 
                 names=['date', 'value']).tail(1)['value']

365    117128.97
Name: value, dtype: float64

### The pd.read_html function, like pd.read_csv, takes a file-like object or a URL. It assumes that it will encounter HTML-formatted text containing at least one table. It turns each table into a data frame and then returns a list of those data frames. With this in mind, retrieve one year of historical S&P 500 data from Yahoo Finance (https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC), looking only at the Date, Close, and Volume columns. Show the date and volume of the days with the highest and lowest Close values. Note that Yahoo seems to look at the User-Agent header in the HTTP request, which cannot be set in read_html. So you’ll need to use requests to retrieve the data, setting User-Agent to a string equal to 'Mozilla 5.0'. Turn the content of the result into a StringIO, and then feed that to read_html and retrieve the data. 

In [83]:
import requests
from io import StringIO


In [78]:
import requests
from bs4 import BeautifulSoup

def fetch_article(url, headers=None):
    """Fetch the HTML content of a Medium article."""
    if headers is None:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://medium.com/',
        'DNT': '1',
    }
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Failed to fetch the article. Status code: {response.status_code}")

def get_title(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract title
    title_tag = soup.find('h1')
    title = title_tag.get_text() if title_tag else 'Untitled'

    return title


if __name__ == '__main__':
    # url = 'https://medium.com/data-engineering-blog/how-to-build-a-data-engineering-blog-with-medium-and-python-733333333333'
    url = 'https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC'
    html_content = fetch_article(url)
    title = get_title(html_content)
    print(f'Title: {title}')
    # print(f'Content: {html_content}')
    df = pd.read_html(StringIO(html_content))
    df2 = pd.read_html(StringIO(html_content))[0].set_index('Date') # .iloc[:-1]


Title: Yahoo Finance


In [72]:
df[0].head()

,Date,Open,High,Low,Close Close price adjusted for splits.,Adj Close Adjusted close price adjusted for splits and dividend and/or capital gain distributions.,Volume
0,"Sep 19, 2025",6647.11,6671.82,6630.31,6664.36,6664.36,5925541000
1,"Sep 18, 2025",6626.85,6656.80,6611.89,6631.96,6631.96,5292400000
2,"Sep 17, 2025",6604.87,6624.39,6551.15,6600.35,6600.35,5805340000
3,"Sep 16, 2025",6624.13,6626.99,6600.11,6606.76,6606.76,5359510000
4,"Sep 15, 2025",6603.49,6619.62,6602.07,6615.28,6615.28,5045020000


In [81]:
df2.iloc[:]

,Open,High,Low,Close Close price adjusted for splits.,Adj Close Adjusted close price adjusted for splits and dividend and/or capital gain distributions.,Volume
Date,,,,,,
"Sep 19, 2025",6647.11,6671.82,6630.31,6664.36,6664.36,5925541000
"Sep 18, 2025",6626.85,6656.80,6611.89,6631.96,6631.96,5292400000
"Sep 17, 2025",6604.87,6624.39,6551.15,6600.35,6600.35,5805340000
"Sep 16, 2025",6624.13,6626.99,6600.11,6606.76,6606.76,5359510000
"Sep 15, 2025",6603.49,6619.62,6602.07,6615.28,6615.28,5045020000
...,...,...,...,...,...,...
"Sep 26, 2024",5762.22,5767.37,5721.01,5745.37,5745.37,4391180000
"Sep 25, 2024",5733.65,5741.03,5712.06,5722.26,5722.26,3624910000
"Sep 24, 2024",5727.66,5735.32,5698.99,5732.93,5732.93,3872710000


In [82]:
df2.iloc[:-1]

,Open,High,Low,Close Close price adjusted for splits.,Adj Close Adjusted close price adjusted for splits and dividend and/or capital gain distributions.,Volume
Date,,,,,,
"Sep 19, 2025",6647.11,6671.82,6630.31,6664.36,6664.36,5925541000
"Sep 18, 2025",6626.85,6656.80,6611.89,6631.96,6631.96,5292400000
"Sep 17, 2025",6604.87,6624.39,6551.15,6600.35,6600.35,5805340000
"Sep 16, 2025",6624.13,6626.99,6600.11,6606.76,6606.76,5359510000
"Sep 15, 2025",6603.49,6619.62,6602.07,6615.28,6615.28,5045020000
...,...,...,...,...,...,...
"Sep 27, 2024",5755.36,5763.78,5727.34,5738.17,5738.17,3898530000
"Sep 26, 2024",5762.22,5767.37,5721.01,5745.37,5745.37,4391180000
"Sep 25, 2024",5733.65,5741.03,5712.06,5722.26,5722.26,3624910000
